In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
print(torch.__version__)

1.10.0+cu111


### Download the dataset before.

In [ ]:

# def load_data_subset(batch_size,
#                      workers,
#                      dataset,
#                      data_target_dir,
#                      labels_per_class=100,
#                      valid_labels_per_class=500,
#                      mixup_alpha=1):
#     '''return datalaoder'''
#     ## copied from GibbsNet_pytorch/load.py
#     if dataset == 'cifar10':
#         mean = [x / 255 for x in [125.3, 123.0, 113.9]]
#         std = [x / 255 for x in [63.0, 62.1, 66.7]]
#     elif dataset == 'cifar100':
#         mean = [x / 255 for x in [129.3, 124.1, 112.4]]
#         std = [x / 255 for x in [68.2, 65.4, 70.4]]
#     elif dataset == 'tiny-imagenet-200':
#         mean = [x / 255 for x in [127.5, 127.5, 127.5]]
#         std = [x / 255 for x in [127.5, 127.5, 127.5]]
#     else:
#         assert False, "Unknow dataset : {}".format(dataset)

#     # pre-processing
#     if dataset == 'tiny-imagenet-200':
#         train_transform = transforms.Compose([
#             transforms.RandomHorizontalFlip(),
#             transforms.RandomCrop(64, padding=4),
#             transforms.ToTensor(),
#             transforms.Normalize(mean, std)
#         ])

#         test_transform = transforms.Compose(
#             [transforms.ToTensor(), transforms.Normalize(mean, std)])
#     else:
#         train_transform = transforms.Compose([
#             transforms.RandomHorizontalFlip(),
#             transforms.RandomCrop(32, padding=2),
#             transforms.ToTensor(),
#             transforms.Normalize(mean, std)
#         ])

#         test_transform = transforms.Compose(
#             [transforms.ToTensor(), transforms.Normalize(mean, std)])

#     # dataset
#     if dataset == 'cifar10':
#         train_data = datasets.CIFAR10(data_target_dir,
#                                       train=True,
#                                       transform=train_transform,
#                                       download=True)
#         test_data = datasets.CIFAR10(data_target_dir,
#                                      train=False,
#                                      transform=test_transform,
#                                      download=True)
#         num_classes = 10
#     elif dataset == 'cifar100':
#         train_data = datasets.CIFAR100(data_target_dir,
#                                        train=True,
#                                        transform=train_transform,
#                                        download=True)
#         test_data = datasets.CIFAR100(data_target_dir,
#                                       train=False,
#                                       transform=test_transform,
#                                       download=True)
#         num_classes = 100
#     elif dataset == 'tiny-imagenet-200':
#         train_root = os.path.join(data_target_dir,
#                                   'train')  # this is path to training images folder
#         validation_root = os.path.join(data_target_dir,
#                                        'val/images')  # this is path to validation images folder
#         train_data = datasets.ImageFolder(train_root, transform=train_transform)
#         test_data = datasets.ImageFolder(validation_root, transform=test_transform)
#         num_classes = 200
#     else:
#         assert False, 'Do not support dataset : {}'.format(dataset)

#     n_labels = num_classes

#     # random sampler
#     def get_sampler(labels, n=None, n_valid=None):
#         # Only choose digits in n_labels
#         # n = number of labels per class for training
#         # n_val = number of lables per class for validation
#         (indices, ) = np.where(reduce(__or__, [labels == i for i in np.arange(n_labels)]))
#         np.random.shuffle(indices)

#         indices_valid = np.hstack([
#             list(filter(lambda idx: labels[idx] == i, indices))[:n_valid] for i in range(n_labels)
#         ])
#         indices_train = np.hstack([
#             list(filter(lambda idx: labels[idx] == i, indices))[n_valid:n_valid + n]
#             for i in range(n_labels)
#         ])
#         indices_unlabelled = np.hstack(
#             [list(filter(lambda idx: labels[idx] == i, indices))[:] for i in range(n_labels)])

#         indices_train = torch.from_numpy(indices_train)
#         indices_valid = torch.from_numpy(indices_valid)
#         indices_unlabelled = torch.from_numpy(indices_unlabelled)
#         sampler_train = SubsetRandomSampler(indices_train)
#         sampler_valid = SubsetRandomSampler(indices_valid)
#         sampler_unlabelled = SubsetRandomSampler(indices_unlabelled)
#         return sampler_train, sampler_valid, sampler_unlabelled

#     if dataset == 'tiny-imagenet-200':
#         pass
#     else:
#         train_sampler, valid_sampler, unlabelled_sampler = get_sampler(
#             train_data.targets, labels_per_class, valid_labels_per_class)

In [ ]:
# from torch.utils.data.sampler import SubsetRandomSampler
# # random sampler
# def get_sampler(labels, n=None, n_valid=None):
#     # Only choose digits in n_labels
#     # n = number of labels per class for training
#     # n_val = number of lables per class for validation
#     (indices, ) = np.where(reduce(__or__, [labels == i for i in np.arange(n_labels)]))
#     np.random.shuffle(indices)

#     indices_valid = np.hstack([
#         list(filter(lambda idx: labels[idx] == i, indices))[:n_valid] for i in range(n_labels)
#     ])
#     indices_train = np.hstack([
#         list(filter(lambda idx: labels[idx] == i, indices))[n_valid:n_valid + n]
#         for i in range(n_labels)
#     ])
#     indices_unlabelled = np.hstack(
#         [list(filter(lambda idx: labels[idx] == i, indices))[:] for i in range(n_labels)])

#     indices_train = torch.from_numpy(indices_train)
#     indices_valid = torch.from_numpy(indices_valid)
#     indices_unlabelled = torch.from_numpy(indices_unlabelled)
#     sampler_train = SubsetRandomSampler(indices_train)
#     sampler_valid = SubsetRandomSampler(indices_valid)
#     sampler_unlabelled = SubsetRandomSampler(indices_unlabelled)
#     return sampler_train, sampler_valid, sampler_unlabelled

In [ ]:
# labels_per_class = 10
# valid_labels_per_class = 10
# sample_size
# train_sampler, valid_sampler, unlabelled_sampler = get_sampler(
#             train_data.targets, labels_per_class, valid_labels_per_class)

In [ ]:
# transform=transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.1307,), (0.3081,))
#     ])
# dataset1 = datasets.FashionMNIST('./data', train=True, download=True,
#                       transform=transform)
# dataset2 = datasets.FashionMNIST('./data', train=False,
#                       transform=transform)

In [ ]:
# import torchvision
# import torch

# ## FashionMNIST
# # trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
# #                                         download=True, transform=None)

# ## CIFAR10
# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                         download=True, transform=None)

# evens = list(range(0, len(trainset), 2))
# odds = list(range(1, len(trainset), 2))
# trainset_1 = torch.utils.data.Subset(trainset, evens)
# trainset_2 = torch.utils.data.Subset(trainset, odds)

# trainloader_1 = torch.utils.data.DataLoader(trainset_1, batch_size=4,
#                                             shuffle=True, num_workers=2)
# trainloader_2 = torch.utils.data.DataLoader(trainset_2, batch_size=4,
#                                             shuffle=True, num_workers=2)


# # dataset1 = datasets.CIFAR10('./data', train=True, download=True,
# #                       transform=transform)
# # dataset2 = datasets.CIFAR10('./data', train=False,
# #                       transform=transform)

In [ ]:
# dataset2.valid_labels_per_class

In [ ]:
import torchvision
import torch
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
evens = list(range(0, len(trainset), 2))
odds = list(range(1, len(trainset), 2))
trainset_1 = torch.utils.data.Subset(trainset, evens)
trainset_2 = torch.utils.data.Subset(trainset, odds)

trainloader_1 = torch.utils.data.DataLoader(trainset_1, batch_size=4,
                                            shuffle=True, num_workers=2)
trainloader_2 = torch.utils.data.DataLoader(trainset_2, batch_size=4,
                                            shuffle=True, num_workers=2)

In [ ]:
quarter = list(range(0, len(trainset), 4))
trainset_quarter = torch.utils.data.Subset(trainset, quarter)

In [ ]:
dataset1 = trainset
dataset2 = testset

### Prepare Toy Convolution Net Architecture

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss_func = nn.CrossEntropyLoss()
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
        # if batch_idx % args.log_interval == 0:
        #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #         epoch, batch_idx * len(data), len(train_loader.dataset),
        #         100. * batch_idx / len(train_loader), loss.item()))
        #     if args.dry_run:
        #         break

In [ ]:
def test(args, model, device, test_loader, epoch):
    model.eval()

    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss_func = nn.CrossEntropyLoss()
            test_loss += loss_func(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    # print('\nTest set: Epoch: {} Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    #     epoch, test_loss, correct, len(test_loader.dataset),
    #     (100. * round(correct / len(test_loader.dataset), 6)))
    
    args.accuracy_list.append(100. * round(correct / len(test_loader.dataset), 6))
    args.loss_list.append(test_loss)
    

### Declare simulated Args Class [contain learning rate and momentum coeff]

In [ ]:
# round(1234/7, 10)

In [ ]:
class SimulatedArgs():
    def __init__(self, epochs, batch_size, optimizer, dry_run, lr_AdaDelta=1, lr_SGD=0.1, momentum=0.9, lr_Adam = 0.01):
        # Training settings
        self.epochs = epochs

        self.batch_size = batch_size  # or 128 [later]
        self.test_batch_size = 1000
        self.no_cuda = False
        
        self.optimizer = optimizer
          # for AdaDelta optimizer
        if (self.optimizer == "AdaDelta"):
          self.lr = lr_AdaDelta
        elif (self.optimizer == "SGD"):
          # for SGD optimizer
          self.lr_SGD = lr_SGD
          self.momentum_SGD = 0.9
        elif (self.optimizer == "Adam"):
          self.lr_Adam = lr_Adam

        self.gamma = 0.7
        self.seed = 1

        self.log_interval = 10
        self.dry_run = dry_run
        self.save_model = False

        # Lists for visualization of loss and accuracy 
        self.loss_list = []
        self.iteration_list = []
        self.accuracy_list = []

        # Lists for knowing classwise accuracy
        self.predictions_list = []
        self.labels_list = []

In [ ]:
def main(args, dataset1, dataset2):
  use_cuda = not args.no_cuda and torch.cuda.is_available()

  torch.manual_seed(args.seed)

  device = torch.device("cuda" if use_cuda else "cpu")

  train_kwargs = {'batch_size': args.batch_size}
  test_kwargs = {'batch_size': args.test_batch_size}
  train_kwargs = {'num_workers': 2}
  test_kwargs = {'num_workers': 2}
  if use_cuda:
      cuda_kwargs = {'num_workers': 1,
                      'pin_memory': True,
                      'shuffle': True}
      train_kwargs.update(cuda_kwargs)
      test_kwargs.update(cuda_kwargs)

  train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
  test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

  model = Net().to(device)

  ## ---------------------------- IMPORTANT ----------------------------------
  # choose correct optimizer
  print('\nTraining SetUp: batch size: {}, optimizer: {} \n'.format(args.batch_size, args.optimizer))
  if (args.optimizer == "AdaDelta"):
    print('Inner Parameters: lr: {} \n'.format(args.lr))
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

  elif (args.optimizer == "SGD"):
    print('Inner Parameters: lr_SGD: {}, momentum_SGD: {} \n'.format(args.lr_SGD, args.momentum_SGD))
    optimizer = optim.SGD(model.parameters(), lr=args.lr_SGD, momentum=args.momentum_SGD)
  elif (args.optimizer == "Adam"):
    print('Inner Parameters: lr_Adam: {} \n'.format(args.lr_Adam))
    optimizer = optim.Adam(model.parameters(), lr=args.lr_Adam)


  # set linear rate updater
  scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

  ## ---------------------------- IMPORTANT ----------------------------------
  
  for epoch in range(1, args.epochs + 1):
      args.iteration_list.append(epoch)
      train(args, model, device, train_loader, optimizer, epoch)
      test(args, model, device, test_loader, epoch)
      scheduler.step()  # To update the learning rates
  
      if (epoch == args.epochs):
          print('\nTest set: Epoch: {} Average loss: {:.4f}, Accuracy: ({:.4f}%)\n'.format(
          epoch, args.iteration_list[-1], args.accuracy_list[-1]))
      
  plt.plot(args.iteration_list, args.loss_list)
  plt.xlabel("No. of Iteration")
  plt.ylabel("Loss")
  plt.title("Iterations vs Loss")
  plt.show()

  plt.plot(args.iteration_list, args.accuracy_list)
  plt.xlabel("No. of Iteration")
  plt.ylabel("Accuracy")
  plt.title("Iterations vs Accuracy")
  plt.show()

  if args.save_model:
      torch.save(model.state_dict(), str(len(dataset1)) + "cifar10_cnn.pt")

# Start Testing!!!

## When the size of training data is relativly close, the accuracy do it as well.

In [ ]:
args = SimulatedArgs(epochs=50, batch_size=128, optimizer = "Adam", dry_run = False)
main(args, dataset1, dataset2)

In [ ]:
# [1] Training SetUp: batch size: 128, optimizer: SGD 
args = SimulatedArgs(epochs=50, batch_size=128, optimizer = "Adam", dry_run = False)
main(args, trainset_1, dataset2)

In [ ]:
args = SimulatedArgs(epochs=50, batch_size=128, optimizer = "Adam", dry_run = False)
main(args, trainset_2, dataset2)

In [ ]:
args = SimulatedArgs(epochs=50, batch_size=128, optimizer = "Adam", dry_run = False)
main(args, trainset_quarter, dataset2)

## When we use full data set --- the improvement of accuracy is obvious